# Workforce Planning Project: Emergency Care Environment

### Overview:

The constant need of nurses in a hospital for many important tasks impart an economic burden on hospital's monetary resources. The problem of optimizing the number of working nurses has two important aspects, first is to predict number of incoming patients and second is planning working shifts of nurses in such a way that number of nurses working at each hour of the day is just enough to satisfy the demand.

In this project we'll focus on the planning part of the problem and will assume the demand for nurses at each hour of the day is known. To do so, we are formulating an Integer Programming problem which will minimize the number of nurse-hours for a day given the constraints.

Model is formulated as follows,

#### A. Indices:
1. i = 1, 2, ... , 32 ==> Hours of the day 
Here, i = 1 to 8 (0 to 7 in python, as index starts from 0) does belong to 17 to 24th hour from past day. While i = 9 to 32 will represent 1 to 24th hour of current day.

2. j = 1, ... , 3 ==> index for different lenghts of shifts

#### B. Parameters, sets, data:
1. s = {}
This set consists of the hours of the day at which nurses cannot start the shift

2. D = []
This array will have demand at each hour of the day

3. shift_length = [4, 6, 8]
This array will have 'j' elements for different allowed length for shift

#### C. Decision variables
$X_{i,j}$, will represent number of nurses starting their shift at the $i^{th}$ hour for $j^{th}$ length

#### D.  Mathematical Model

\begin{equation}
Min \sum_{i} (4X_{i,0} + 6X_{i,1} + 8X_{i,2})
\end{equation}

Such that,
\begin{equation}
\sum_{j} X_{i,j} + \sum_{m = 1}^{3} X_{i - m,0} + \sum_{n = 1}^{5} X_{i - n,1} + \sum_{o = 1}^{7} X_{i - o, 2} >= D_{i}
\end{equation}

\begin{equation}
X_{i,j} = X_{i + 24, j}
\end{equation}

\begin{equation}
X_{s,j} = 0
\end{equation}

$X_{i,j} >= 0 \forall i, j$

In [2]:
# import required libraries
from gurobipy import *
import sys
import numpy as np
from collections import defaultdict

# defining parameters
shift_length = [4,6,8]      # these are the available options for shift length
Demand = [19,16,12,10,7,6,6,6,8,10,14,18,22,23,23,22,22,21,21,21,21,22,21,21]
start_deny = []

In Gurobi, we are required to define the model before we start to formulate the problem.

In [6]:
# function for defining empty model

def get_empty_model():
    m = Model()
    
    return m

As we have defined an empty model now, we can proceed further by defining each element of LP formulation and adding it to the model, i.e.

1. Decision variables
2. Constraints 
3. Objective function 

Lets start by defining decision varibles first

In [7]:
# Function for defining variables according to the given parameters

def get_variables(m, shift_length, Demand):
    
    # decision varible is number of nurses working at a specific hour for specific length
    '''
    x = m.addVars(len(start_hours) + max(shift_length), len(shift_length), 
                  lb = 0, vtype=GRB.INTEGER)
    '''
    x = {}
    for i in range(len(Demand) + max(shift_length)):
        for j in range(len(shift_length)):
            if (i < max(shift_length)):
                x[(i,j)] = m.addVar(lb = 0, ub = Demand[24 - max(shift_length) + i],
                                    vtype=GRB.INTEGER, 
                                    name = 'Yesterday: Hour = %d, Lenght = %d'%(24 - max(shift_length) + i + 1, shift_length[j]))
            else:
                x[(i,j)] = m.addVar(lb = 0, ub = Demand[i - max(shift_length)],
                                    vtype=GRB.INTEGER, 
                                    name = 'Today: Hour = %d, Lenght = %d'%(i + 1 - max(shift_length), shift_length[j]))
    
    x = tupledict(x)
    #print(x.keys())
    
    m.update()
    return m,x

In [8]:
# TEST CELL

m = get_empty_model()
m,x = get_variables(m, shift_length, Demand)

In [10]:
# In this cell we'll define function fwhich will return all constraints

def get_constraint(m, x, shift_length, Demand, start_deny):
    
    # Constraint - 1:
    # we have variables defined for 32 hours, imagine the midnight hour starting at 9th row,
    # so the variables defined in first 8 rows represents yesterday's number of nurses, which
    # should be equal to variables defined in the rows 25 to 32, which is essentially the same
    # as vars defined in 1 to 8 for next day.
    c1 = m.addConstrs(x[i, j] == x[i + 24, j] for j in range(len(shift_length))
                                              for i in range(max(shift_length)))
            
    #print(c1.keys())
    
    # Constraint - 2:
    # number of nurses working at each hour of the day should satisfy the demand of that specific hour
    c21 = {}
    for s_h in range(max(shift_length), 24 + max(shift_length)):  
        c21[s_h] = m.addConstr(quicksum(x[i,0] for i in range(s_h - shift_length[0] + 1, s_h + 1)) +
                               quicksum(x[j,1] for j in range(s_h - shift_length[1] + 1, s_h + 1)) +
                               quicksum(x[k,2] for k in range(s_h - shift_length[2] + 1, s_h + 1)) -
                               Demand[s_h - max(shift_length)] >= 0)
        
    c22 = {}
    for s_h in range(max(shift_length), 24 + max(shift_length)):  
        c22[s_h] = m.addConstr(quicksum(x[i,0] for i in range(s_h - shift_length[0] + 1, s_h + 1)) +
                               quicksum(x[j,1] for j in range(s_h - shift_length[1] + 1, s_h + 1)) +
                               quicksum(x[k,2] for k in range(s_h - shift_length[2] + 1, s_h + 1)) -
                               Demand[s_h - max(shift_length)] <= 10)
    #print(c2.keys())
                  

    # Constraint - 2:
    # if we put constraint on starting hour then and then only this constraint will be active
    c3 = {}
    if (len(start_deny) != 0):
        set_deny = (start_deny)
        c3 = m.addConstrs(x[i - 1 + max(shift_length), j] == 0 
                          for i in set_deny
                          for j in range(len(shift_length)))
    
        #print(c3.keys())      
 
    con = {
        "var equality con": c1, 
        "demand con1"     : c21,
        "demand con2"     : c22,
        "start deny con"  : c3
    }
    m.update()
    return m, con
        

In [11]:
# TEST CELL

m = get_empty_model()
m,x = get_variables(m, shift_length, Demand)
m = get_constraint(m, x, shift_length, Demand, [4, 8, 10])

We have defined functions which will return decision variables and constraints in LP formulation. Now lets define the objective functio, call the variables and constraints and solve the LP model

In [17]:
# In this cell we'll define objective function for the optimization problem
def get_objective(m, x, con, Demand):
    
    # here we are defining cost-coefficients for each decision variable
    coeff = {}
    for i in range(len(Demand) + max(shift_length)):
        for j in range(len(shift_length)):
            coeff[(i,j)] = shift_length[j]
    #print(coeff)
    
    # setting the objective
    m.setObjective(x.prod(coeff), GRB.MINIMIZE)
    #m.setObjective(x.sum(), GRB.MINIMIZE)

    # update model
    m.update()
    
    # solve optimization problem
    m.optimize()
    
    # print attributes
    m.printAttr('x')
    
    return m

In [18]:
# In this cell we'll solve the optimization problem for the scenario 1
# Scenario 1: Nurses can start shift at any hour of the day, therefore start_deny = []

# define empty model
m = get_empty_model()

# call decision var
m, x = get_variables(m, shift_length, Demand)

# call constraints
start_deny = []
m, con = get_constraint(m, x, shift_length, Demand, start_deny)

# solve optimization problem
m = get_objective(m, x, con, Demand)

Optimize a model with 72 rows, 96 columns and 912 nonzeros
Variable types: 0 continuous, 96 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 8e+00]
  Bounds range     [6e+00, 2e+01]
  RHS range        [6e+00, 3e+01]
Found heuristic solution: objective 636.0000000
Presolve removed 24 rows and 24 columns
Presolve time: 0.00s
Presolved: 48 rows, 72 columns, 864 nonzeros
Variable types: 0 continuous, 72 integer (0 binary)

Root relaxation: objective 4.760000e+02, 36 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  476.00000    0    9  636.00000  476.00000  25.2%     -    0s
H    0     0                     482.0000000  476.00000  1.24%     -    0s
H    0     0                     478.0000000  476.00000  0.42%     -    0s
     0     0  476.00000    0   14  478.00000  476.00000  0.42%     -    0s
*

In [19]:
# In this cell we'll solve the optimization problem for the scenario 2
# Scenario 2: Nurses can start shift only at 1st, 5th, 9th, 13th, 17th and 21st hour of the day, 
# therefore start_deny defined as follows

# define empty model
m = get_empty_model()

# call decision var
m, x = get_variables(m, shift_length, Demand)

# call constraints
start_deny = [2, 3, 4, 6, 7, 8, 10, 11, 12, 14, 15, 16, 18, 19, 20, 22, 23, 24]
m, con = get_constraint(m, x, shift_length, Demand, start_deny)

# solve optimization problem
m = get_objective(m, x, con, Demand)

Optimize a model with 126 rows, 96 columns and 966 nonzeros
Variable types: 0 continuous, 96 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 8e+00]
  Bounds range     [6e+00, 2e+01]
  RHS range        [6e+00, 3e+01]
Found heuristic solution: objective 676.0000000
Presolve removed 110 rows and 79 columns
Presolve time: 0.00s
Presolved: 16 rows, 17 columns, 70 nonzeros
Variable types: 0 continuous, 17 integer (0 binary)

Root relaxation: objective 5.320000e+02, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     532.0000000  532.00000  0.00%     -    0s

Explored 0 nodes (14 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 532 676 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.320000000000e+02, best bound 5